In [1]:
import os
import re

import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np

from google.cloud.bigquery import Client

Path = '/Users/georgiinusuev/PycharmProjects/SurgeVSLongRides/2comp/choose_cities'

In [2]:
start_date = '2024-10-11 00:00'
end_date='2024-11-11 00:00'

formatted_start_date = dt.datetime.strptime(start_date, '%Y-%m-%d %H:%M').strftime('%b %d, \'%y')
formatted_end_date = dt.datetime.strptime(end_date, '%Y-%m-%d %H:%M').strftime('%b %d, \'%y')

In [3]:
def get_data(start_date='2024-11-16 00:00', end_date='2024-12-16 00:00'):
    client = Client(project='analytics-dev-333113')

    query = f'''
    WITH incity_detail AS (SELECT order_uuid,
                                  city_id,
                                  order_type,
                                  created_date_order_part,
                                  driverdone_timestamp                                 AS driverdone_dttm,
                                  price_order_usd,
                                  price_highrate_usd,
                                  price_start_usd,
                                  CAST(FLOOR(distance_in_meters / 300) * 300 AS INT64) AS bin_start
                           FROM `indriver-e6e40.emart.incity_detail`
                           WHERE true
    --                          AND (order_type, city_id) in ()
                             AND created_date_order_part BETWEEN DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '{start_date}')) AND
                               DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '{end_date}'))
                             AND order_uuid IS NOT NULL
                             AND city_id IS NOT NULL
                             AND driverdone_timestamp IS NOT NULL),
    
         order_global AS (SELECT DISTINCT uuid,                  -- order_uuid from incity_detail
                                          price_calculation_uuid -- id from pricing_logs
                          FROM `indriver-e6e40.ods_new_order_rh_cdc.order_global_strm`
                          WHERE true
                            AND DATE(TIMESTAMP_TRUNC(created_at, DAY)) BETWEEN DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '{start_date}')) AND
                              DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '{end_date}'))
                            AND uuid IS NOT NULL
                            AND price_calculation_uuid IS NOT NULL),
    
         pricing_logs AS (SELECT t1.id            AS price_log_id,
                                 t1.dynamic_surge AS dynamic_surge
                          FROM `indriver-e6e40.ods_recprice_cdc.pricing_logs` AS t1
                          WHERE true
                            AND DATE(TIMESTAMP_TRUNC(calculation_dttm, DAY)) BETWEEN DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '{start_date}')) AND
                              DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '{end_date}'))
                            AND t1.id IS NOT NULL
                            AND calculator_id IS NOT NULL),
    
         total as (SELECT city_id,
                          order_type,
                          bin_start,
                          count(*) AS frequency,
                          AVG(IF(dynamic_surge = 1, price_order_usd, NULL)) /
                          AVG(IF(dynamic_surge = 1, price_highrate_usd, NULL)) - 1 AS Done2Rec_NOTsurged,
                          AVG(IF((dynamic_surge BETWEEN 0.95 AND 1.05) AND dynamic_surge != 1, price_order_usd, NULL)) /
                          AVG(IF((dynamic_surge BETWEEN 0.95 AND 1.05) AND dynamic_surge != 1, price_highrate_usd, NULL)) - 1 AS Done2Rec_NOTsurged_nbrhd,
                          AVG(IF(dynamic_surge > 1, price_order_usd, NULL)) /
                          AVG(IF(dynamic_surge > 1, price_highrate_usd, NULL)) - 1 AS Done2Rec_surged                      
                   FROM incity_detail AS incity
                            LEFT JOIN order_global AS order_global
                                      ON incity.order_uuid = order_global.uuid
                            LEFT JOIN pricing_logs AS pricing_logs
                                      ON order_global.price_calculation_uuid = pricing_logs.price_log_id
                   GROUP BY city_id, order_type, bin_start)
    
    SELECT total.*,
           ABS(Done2Rec_NOTsurged - Done2Rec_surged) AS Done2Rec_diff,
           ABS(Done2Rec_NOTsurged_nbrhd - Done2Rec_surged) AS Done2Rec_nbrhd_diff,
           rec_cnt / rides AS rec_share,
           stats.rides,
           stats.surged_rides_share
    FROM total
             left join (SELECT city_id,
                               order_type,
                               COUNT(order_uuid)                                                  AS rides,
                               COUNT(IF(dynamic_surge > 1, order_uuid, NULL)) / COUNT(order_uuid) AS surged_rides_share,
                               count(if(SAFE_DIVIDE(price_highrate_usd, price_start_usd) - 1 <= 0.1, order_type, NULL)) AS rec_cnt,
                        FROM incity_detail AS incity
                                 LEFT JOIN order_global AS order_global
                                           ON incity.order_uuid = order_global.uuid
                                 LEFT JOIN pricing_logs AS pricing_logs
                                           ON order_global.price_calculation_uuid = pricing_logs.price_log_id
                        GROUP BY city_id, order_type) as stats
                       on total.city_id = stats.city_id and total.order_type = stats.order_type
    WHERE rides > 20000
    ORDER BY surged_rides_share, order_type, bin_start;
    '''
    
    print(query)
    df = client.query(query).to_dataframe()
    return df

In [4]:
def binned_percentile(df, x, bin_start='bin_start', frequency='frequency'):
    data = df.reset_index(drop=True)

    data['cumulative_frequency'] = data[frequency].cumsum()

    total_frequency = data[frequency].sum()

    percentile_freq = x * total_frequency

    percentile_bin = data[data['cumulative_frequency'] >= percentile_freq].iloc[0]

    bin_index = data.index[data['cumulative_frequency'] >= percentile_freq][0]
    previous_cumulative = data.iloc[bin_index - 1]['cumulative_frequency'] if bin_index > 0 else 0
    previous_bin_start = data.iloc[bin_index - 1][bin_start] if bin_index > 0 else 0
    current_bin_start = percentile_bin[bin_start]
    current_frequency = percentile_bin[frequency]

    excess = percentile_freq - previous_cumulative
    bin_range = current_bin_start - previous_bin_start
    percentile_value = previous_bin_start + (excess / current_frequency) * bin_range

    return percentile_value


def sanitize_filename(name):
    return re.sub(r'[\\/:"*?<>|]+', '_', name)


def draw(df, surged_share_min=0.1, dist_bin_min=300, dist_bin_max_perc=0.9999, dist_bin_max_filter_perc=0.99):
    data = df
    save_dir = Path + '/images'
    stats = []

    data = data[data['surged_rides_share'] >= surged_share_min]

    for city_id in data['city_id'].unique():
        for order_type in data[data['city_id'] == city_id]['order_type'].unique():
            city_data_og = data[(data['city_id'] == city_id) & (data['order_type'] == order_type)]
            fig, axes = plt.subplots(3, 1, figsize=(10, 20))

            min_bin = dist_bin_min
            max_bin = binned_percentile(city_data_og, dist_bin_max_perc)
            city_data_unfiltered = city_data_og[(city_data_og['bin_start'] <= max_bin) & (city_data_og['bin_start'] >= min_bin)]
            max_bin = binned_percentile(city_data_og, dist_bin_max_filter_perc)
            city_data = city_data_og[(city_data_og['bin_start'] <= max_bin) & (city_data_og['bin_start'] >= min_bin)]

            if city_data.empty:
                continue

            rides = city_data['rides'].min()
            surged_share = city_data['surged_rides_share'].min()

            # Plot the data         
            axes[0].plot(city_data_unfiltered['bin_start'], city_data_unfiltered['Done2Rec_surged'], marker='o', linestyle='-', label=f'Surged')
            axes[0].plot(city_data_unfiltered['bin_start'], city_data_unfiltered['Done2Rec_NOTsurged'], marker='o', linestyle='-', label=f'Not Surged')
            axes[0].plot(city_data_unfiltered['bin_start'], city_data_unfiltered['Done2Rec_NOTsurged_nbrhd'], marker='o', linestyle='-', label=f'Not Surged NBRHD')
            axes[1].plot(city_data['bin_start'], city_data['Done2Rec_surged'], marker='o', linestyle='-', label=f'Surged')
            axes[1].plot(city_data['bin_start'], city_data['Done2Rec_NOTsurged'], marker='o', linestyle='-', label=f'Not Surged')
            axes[1].plot(city_data['bin_start'], city_data['Done2Rec_NOTsurged_nbrhd'], marker='o', linestyle='-', label=f'Not Surged NBRHD')
            axes[2].plot(city_data['bin_start'], city_data['Done2Rec_diff'], marker='o', linestyle='-', label=f'Difference')
            axes[2].plot(city_data['bin_start'], city_data['Done2Rec_nbrhd_diff'], marker='o', linestyle='-', label=f'Difference 4NBRHD')
            
            # Add linear regression line for Done2Rec_diff
            x = city_data['bin_start']
            y = city_data['Done2Rec_diff']
            min_y_index = y.idxmin()
            min_x = x.loc[min_y_index]
            mask = x >= min_x
            x_filtered = x[mask]
            y_filtered = y[mask]
            
            ratio = len(x_filtered) / len(x) if len(x) > 0 else 0
            stats.append({'city_id': city_id, 'order_type': order_type, 'ratio': ratio})
            
            if len(x_filtered) > 1:  
                slope, intercept = np.polyfit(list(x_filtered), list(y_filtered), 1)  
                reg_line = slope * x_filtered + intercept
                axes[2].plot(x_filtered, reg_line, color='red', linestyle='--', label='Regression Line')
                
            # Add vertical lines for percentiles
            for ax in axes:
                for perc in [0.25, 0.50, 0.75, 0.99]:
                    bin_line = binned_percentile(city_data_og, perc)
                    ax.axvline(bin_line, linestyle='--', color='grey', alpha=0.7)
                    ax.text(bin_line, ax.get_ylim()[0], f'{perc:.0%}', rotation=0, verticalalignment='bottom', fontsize=8)

            for ax in axes:
                # Add plot labels and title
                ax.set_xlabel('Bin Start, meters')
                ax.set_ylabel('Ratio')
                ax.set_title(f'{city_id} - {order_type}, Ratio vs. Distance')

                # Add text annotations
                ax.text(0.05, 0.95,
                        f'Rides: {rides:,}\nSurged Share: {surged_share:.0%}\nDate Range: {formatted_start_date} - {formatted_end_date}',
                        transform=ax.transAxes, fontsize=10, verticalalignment='top')

                ax.set_xlim(0, ax.get_lines()[len(ax.get_lines()) - len(ax.get_lines())].get_xdata().max())
                ax.grid(True)
                ax.legend(fancybox=False, shadow=False)

            sanitized_order_type = sanitize_filename(order_type)
            save_path = os.path.join(save_dir, f'{city_id}_{sanitized_order_type}.png')
            fig.savefig(save_path)
            plt.close(fig)

    print("Plots saved successfully.")
    return stats

In [5]:
Path = '/Users/georgiinusuev/PycharmProjects/SurgeVSLongRides/2comp/choose_cities'
df = get_data()
stts = draw(df)


    WITH incity_detail AS (SELECT order_uuid,
                                  city_id,
                                  order_type,
                                  created_date_order_part,
                                  driverdone_timestamp                                 AS driverdone_dttm,
                                  price_order_usd,
                                  price_highrate_usd,
                                  price_start_usd,
                                  CAST(FLOOR(distance_in_meters / 300) * 300 AS INT64) AS bin_start
                           FROM `indriver-e6e40.emart.incity_detail`
                           WHERE true
    --                          AND (order_type, city_id) in ()
                             AND created_date_order_part BETWEEN DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '2024-11-16 00:00')) AND
                               DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '2024-12-16 00:00'))
                             AND order_uuid IS NOT 

In [9]:
Path = '/Users/georgiinusuev/PycharmProjects/SurgeVSLongRides/2comp/choose_cities/beforeexp2comp'
df = get_data(start_date='2024-10-11 00:00', end_date='2024-11-11 00:00')
stts = draw(df[(df['city_id'].isin([4197, 4227, 4231])) & (df['order_type'] == 'auto_econom')])
stts.sort(key=lambda x: x['ratio'], reverse=True)
stts


    WITH incity_detail AS (SELECT order_uuid,
                                  city_id,
                                  order_type,
                                  created_date_order_part,
                                  driverdone_timestamp                                 AS driverdone_dttm,
                                  price_order_usd,
                                  price_highrate_usd,
                                  price_start_usd,
                                  CAST(FLOOR(distance_in_meters / 300) * 300 AS INT64) AS bin_start
                           FROM `indriver-e6e40.emart.incity_detail`
                           WHERE true
    --                          AND (order_type, city_id) in ()
                             AND created_date_order_part BETWEEN DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '2024-10-11 00:00')) AND
                               DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '2024-11-11 00:00'))
                             AND order_uuid IS NOT 

[{'city_id': 4231, 'order_type': 'auto_econom', 'ratio': 0.9859154929577465},
 {'city_id': 4197, 'order_type': 'auto_econom', 'ratio': 0.8863636363636364},
 {'city_id': 4227, 'order_type': 'auto_econom', 'ratio': 0.8073394495412844}]

In [171]:
df = get_data(start_date='2024-10-11 00:00', end_date='2024-11-11 00:00')
df = df[(df['city_id'].isin([4197, 4227, 4231])) & (df['order_type'] == 'auto_econom')]
df.head()


    WITH incity_detail AS (SELECT order_uuid,
                                  city_id,
                                  order_type,
                                  created_date_order_part,
                                  driverdone_timestamp                                 AS driverdone_dttm,
                                  price_order_usd,
                                  price_highrate_usd,
                                  price_start_usd,
                                  CAST(FLOOR(distance_in_meters / 300) * 300 AS INT64) AS bin_start
                           FROM `indriver-e6e40.emart.incity_detail`
                           WHERE true
    --                          AND (order_type, city_id) in ()
                             AND created_date_order_part BETWEEN DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '2024-10-11 00:00')) AND
                               DATE(PARSE_TIMESTAMP('%Y-%m-%d %H:%M', '2024-11-11 00:00'))
                             AND order_uuid IS NOT 

,city_id,order_type,bin_start,frequency,Done2Rec_NOTsurged,Done2Rec_surged,Done2Rec_diff,rec_share,rides,surged_rides_share
0,5681,6-seater XL,<NA>,736,-0.007246,NaN,NaN,0.913183,28324,0.0
1,5681,6-seater XL,0,356,0.015683,NaN,NaN,0.913183,28324,0.0
2,5681,6-seater XL,300,173,0.117056,NaN,NaN,0.913183,28324,0.0
3,5681,6-seater XL,600,391,0.074219,NaN,NaN,0.913183,28324,0.0
4,5681,6-seater XL,900,476,0.070855,NaN,NaN,0.913183,28324,0.0


In [174]:
df = df[(df['city_id'].isin([4197, 4227, 4231])) & (df['order_type'] == 'auto_econom')]

for city_id in df['city_id'].unique():
    for order_type in df[df['city_id'] == city_id]['order_type'].unique():
        print(city_id, order_type,
              df[(df['city_id'] == city_id) & (df['order_type'] == order_type)]['rec_share'].max(),
              df[(df['city_id'] == city_id) & (df['order_type'] == order_type)]['rec_share'].min())

4227 auto_econom 0.6788039927969279 0.6788039927969279
4231 auto_econom 0.723909897362251 0.723909897362251
4197 auto_econom 0.6363913420274022 0.6363913420274022
